In [4]:
import pymongo
import bson
import utm
import pandas as pd
from pymongo import MongoClient
import time

t1 = time.time()

with open('/Users/mac/Desktop/dump3/test/buildings.bson','rb') as f:
    buildings = bson.decode_all(f.read())
with open('/Users/mac/Desktop/dump3/test/aps.bson','rb') as f:
    aps = bson.decode_all(f.read())
with open('/Users/mac/Desktop/dump3/test/wifi.bson','rb') as f:
    wifi = bson.decode_all(f.read())
with open('/Users/mac/Desktop/dump3/test/paths.bson','rb') as f:
    paths = bson.decode_all(f.read())


    
aps_df = pd.DataFrame(aps)
buildings_df = pd.DataFrame(buildings)
wifi_df = pd.DataFrame(wifi)
paths_df = pd.DataFrame(paths)
buildings_NC_df = buildings_df[(buildings_df['CampusCode']=='North Campus')
                               &(buildings_df['GeographicCampus']=='North Campus')]

list_path = []
list_path_full = []
for path in paths:
    list_temp = []
    list_temp_full = []
    for temp in path['readings']:
        point = temp['connectedAP']['apName']
        try:
            lat = next((l for l in aps if l['apName'] == point), None)['GIS']['x']
            lon = next((l for l in aps if l['apName'] == point), None)['GIS']['y']
        except TypeError:
            pass
        if (lat,lon) not in list_temp:
            list_temp.append((lat,lon))
            temp.update({'t_end':temp['timestamp']})
            list_temp_full.append(temp)
            t_start = temp['timestamp']
        else:
            t_newend = temp['timestamp']
            list_temp_full[-1].update({'t_end':t_newend})
            
    list_path.append(list_temp)
    list_path_full.append(list_temp_full)

t2 = time.time()
    
path_new = [t for t in list_path if len(t) > 1]

print('time cost is:' + str(t2 - t1))

time cost is:163.95279812812805


In [5]:
path_new_full = [t for t in list_path_full if len(t) > 0]

In [6]:
list_path_df = []
for path in path_new_full:
    df1 = pd.DataFrame(path)
    df1['lon_gps'] = pd.Series(0,index = df1.index)
    df1['lat_gps'] = pd.Series(0,index = df1.index)
    df1['lon_utm'] = pd.Series(0,index = df1.index)
    df1['lat_utm'] = pd.Series(0,index = df1.index)
    i = 0
    while i < len(df1.index):
        tmp1 = df1.loc[i,'connectedAP']['apName']
        tmp2 = aps_df['apName']==tmp1
        tmp3 = aps_df[tmp2]
        if tmp3.size > 0:
            tmp4 = tmp3.iloc[0]
            tmp5 = tmp4['GIS']
            #df1.loc[i,'x'] = utm.from_latlon(float(tmp5['y']),float(tmp5['x']))[0]
            df1.loc[i,'lon_gps'] = tmp5['x']
            df1.loc[i,'lat_gps'] = tmp5['y']
            df1.loc[i,'lon_utm'] = utm.from_latlon(float(tmp5['y']),float(tmp5['x']))[0]
            df1.loc[i,'lat_utm'] = utm.from_latlon(float(tmp5['y']),float(tmp5['x']))[1] 
        i = i + 1
    list_path_df.append(df1)

In [7]:
x_min = 275766
x_max = 278141
y_min = 4685200
y_max = 4687178
trip = []
for path_tmp in list_path_df:
        if (min(path_tmp['lon_utm'])>x_min)&(max(path_tmp['lon_utm'])<x_max)&(min(path_tmp['lat_utm'])>y_min)&(max(path_tmp['lat_utm'])<y_max):
            trip.append(path_tmp)

In [10]:
%matplotlib
import geopandas as gpd
import glob
from itertools import tee
import networkx as nx
import pandas as pd
import numpy as np
from shapely.geometry import Point, Polygon, LineString
import matplotlib.pyplot as plt
import matplotlib
import json
import os
from collections import defaultdict
from shapely.geometry import Point, Polygon, LineString

Roads = gpd.read_file("/Users/mac/Desktop/utm_test/Roads_new_line.shp")

def pairwise(line_coords):
    a,b = tee(line_coords, 2)
    next(b, None)
    return zip(a,b) #return (p1,p2), (p2,p3)...

def create_g(Geo_dataFrame):
    G = nx.DiGraph()
    for line in Geo_dataFrame.geometry:
       seq = pairwise(line.coords)
       for p1, p2 in seq:
           G.add_edge(p1, p2)
    return G

#Euclidean distance
def dist(n0, n1):
    x1, y1 = n0
    x2, y2 = n1
    distance = np.sqrt((x2-x1)**2 + (y2-y1)**2)
    return distance

def closest_node(sub_graph, start, end): 
    start_delta = float("inf")
    end_delta = float("inf")
    for n in sub_graph.nodes():
        s_dist = dist(start, n)
        e_dist = dist(end, n) 
        if s_dist < start_delta:
            nn_start = n
            start_delta = s_dist
        if e_dist < end_delta:
            nn_end = n
            end_delta = e_dist
    return (nn_start, nn_end)

def add_attr(sub_graph):
    for n0, n1 in sub_graph.edges_iter():
        distance = dist(n0, n1)
        sub_graph[n0][n1]["dist"] = distance

def shortest_dist(sub_graph, nn_start, nn_end):
    shortest_distance = nx.shortest_path_length(sub_graph, source= nn_start ,target= nn_end, weight= "dist")
    return shortest_distance
def shortest_path(sub_graph, nn_start, nn_end):
    shortest_path = nx.shortest_path(sub_graph, source= nn_start, target= nn_end, weight= "dist")
    return shortest_path
def calculate_distance(origin_coords, dest_coords, graph):
    nn_start, nn_end = closest_node(graph, origin_coords, dest_coords)
    shortest_distance = shortest_dist(graph, nn_start, nn_end)
    return shortest_distance

def calculate_path(origin_coords, dest_coords, graph):
    nn_start, nn_end = closest_node(graph, origin_coords, dest_coords)
    optimal_path = shortest_path(graph, nn_start, nn_end)
    return optimal_path

#convert road into graph
G = create_g(Roads)
sg = list(nx.connected_component_subgraphs(G.to_undirected()))[0]
add_attr(sg) #add distance attribute

Using matplotlib backend: MacOSX


In [11]:
#find path in trip which contains more than one point to calculate the distance of trip
list_trip_df = []
for path_df in trip:
    if len(path_df.index)>1:
        list_trip_df.append(path_df)

In [22]:
list_trip_df[0]

,_id,connectedAP,ip,mac,rssi,session,signalNoiseRatio,source,ssid,t_end,timestamp,user,lon_gps,lat_gps,lon_utm,lat_utm
0,5968f5b278c2782be07cd482,"{'radioName': None, 'apName': 'DC-1F48-S'}",35.2.139.73,557b4c5d73b1d4e9d61b4239e4606b9f1de67c526086b5...,None,{'id': '5fc7b292f1bd31b2ff149dc28e702a120cb24f...,None,RADIUS,MWireless,2017-07-14 15:51:03,2017-07-14 15:47:05,{'uniqname': 'fdc1453c7ad1c2bb542d28b9cdd13486...,-83.71579239,42.29114972,276104.658225,4.685675e+06
1,5968f5b078c2782be07cab12,"{'radioName': None, 'apName': 'NCCOMM-1C01'}",35.2.67.4,557b4c5d73b1d4e9d61b4239e4606b9f1de67c526086b5...,None,{'id': '05b05e2cf6bfd2cc67c0d3290c980fe8fed56f...,None,RADIUS,MWireless,2017-07-14 16:39:29,2017-07-14 15:51:10,{'uniqname': 'fdc1453c7ad1c2bb542d28b9cdd13486...,-83.7174755,42.29113561,275965.839271,4.685678e+06


In [26]:
lat_line = 4686090
#add t dist eudist to each trip dataframe
t3 = time.time()
for temp in list_trip_df:
    temp['delta_t'] = pd.Series(0, index = temp.index)
    temp['delta_sec'] = pd.Series(0, index = temp.index)
    temp['delta_dist'] = pd.Series(0, index = temp.index)
    i = 1
    while i < len(temp.index):
        temp.loc[i,'delta_t'] = temp.loc[i,'timestamp'] - temp.loc[i-1,'t_end']
        temp.loc[i,'delta_sec'] = temp.loc[i,'delta_t'].total_seconds()
        N1 = temp.loc[i,'lon_utm'],temp.loc[i,'lat_utm']
        N2 = temp.loc[i-1,'lon_utm'],temp.loc[i-1,'lat_utm']
        if max(N1[1],N2[1]) < lat_line:
            temp.loc[i,'delta_dist'] = dist(N1,N2)
        else: 
            temp.loc[i,'delta_dist'] = calculate_distance(N2,N1,sg)
        i = i + 1
t4 = time.time()
print('time cost is:' + str(t4-t3))

time cost is:8.389981031417847


In [307]:
list_trip_df[0]

,_id,connectedAP,ip,mac,rssi,session,signalNoiseRatio,source,ssid,t_end,timestamp,user,lon_gps,lat_gps,lon_utm,lat_utm,delta_t,delta_sec,delta_dist
0,5968f5b278c2782be07cd482,"{'radioName': None, 'apName': 'DC-1F48-S'}",35.2.139.73,557b4c5d73b1d4e9d61b4239e4606b9f1de67c526086b5...,None,{'id': '5fc7b292f1bd31b2ff149dc28e702a120cb24f...,None,RADIUS,MWireless,2017-07-14 15:51:03,2017-07-14 15:47:05,{'uniqname': 'fdc1453c7ad1c2bb542d28b9cdd13486...,-83.71579239,42.29114972,276104.658225,4.685675e+06,0,0.0,0.000000
1,5968f5b078c2782be07cab12,"{'radioName': None, 'apName': 'NCCOMM-1C01'}",35.2.67.4,557b4c5d73b1d4e9d61b4239e4606b9f1de67c526086b5...,None,{'id': '05b05e2cf6bfd2cc67c0d3290c980fe8fed56f...,None,RADIUS,MWireless,2017-07-14 16:39:29,2017-07-14 15:51:10,{'uniqname': 'fdc1453c7ad1c2bb542d28b9cdd13486...,-83.7174755,42.29113561,275965.839271,4.685678e+06,0 days 00:00:07,7.0,138.848493


In [275]:
def walktime(d):
    walkspeed = 1.5 #m/s
    walktime  = d/walkspeed
    variance = 0.3*walktime
    t_max = walktime+variance
    t_min = walktime-variance
    return t_min,t_max

def drivetime(d):
    drivespeed = 11 #m/s
    drivetime  = d/drivespeed
    variance = 0.3*drivetime
    t_max = drivetime+variance
    t_min = drivetime-variance
    return t_min,t_max

In [276]:
list_delta_sec = []
list_dist = []

for tmp in list_trip_df:
    sec_tmp = tmp['delta_sec'].tolist()
    dist_tmp = tmp['delta_dist'].tolist()
    del sec_tmp[0]
    del dist_tmp[0]
    list_delta_sec = list_delta_sec + sec_tmp
    list_dist = list_dist + dist_tmp

In [277]:
walk_sec = []
walk_dist = []
drive_sec = []
drive_dist = []
i = 0
while i < len(list_dist):
    if walktime(list_dist[i])[0] <= list_delta_sec[i] <= walktime(list_dist[i])[1]:
        walk_sec.append(list_delta_sec[i])
        walk_dist.append(list_dist[i])
    elif drivetime(list_dist[i])[0] <= list_delta_sec[i] <= drivetime(list_dist[i])[1]:
        drive_sec.append(list_delta_sec[i])
        drive_dist.append(list_dist[i])
    i = i + 1

In [336]:
len(drive_sec)

203

In [316]:
# select those trips which satisfy walk/drive constraints------------------------
walk_trip = []    # list of trips which satisfy walk constraints
drive_trip = []   # list of trips which satisfy drive constraints
for tmp in list_trip_df:
    for i in tmp.index(tmp.index!=0):
        t_tmp = tmp.loc[i,'delta_sec']

ValueError: Can only compare identically-labeled DataFrame objects

In [334]:
list_trip_df[3].loc[]

_id                                          5968f5b078c2782be07ca501
connectedAP              {'radioName': None, 'apName': 'NCCOMM-1202'}
ip                                                       35.2.122.190
mac                 c6743fa4f59bd0c16c9e0a4e1eba52c934a35e602aaa25...
rssi                                                             None
session             {'id': 'b660368835aa3a89be3c73ea761b9f83fcab73...
signalNoiseRatio                                                 None
source                                                         RADIUS
ssid                                                        MWireless
t_end                                             2017-07-14 15:57:41
timestamp                                         2017-07-14 15:51:45
user                {'uniqname': 'b8b05456adf11c95d63bd3553994f1af...
lon_gps                                                   -83.7174755
lat_gps                                                   42.29113561
lon_utm             

In [273]:
#plot regression-------------------------------------------
plt.scatter(list_delta_sec,list_dist,color='blue',s=1)
plt.scatter(walk_sec,walk_dist,color='orange',s=3)
plt.scatter(drive_sec,drive_dist,color='green',s=3)
plt.plot(np.arange(0,1500),1.5*np.arange(0,1500),'r--')
plt.plot(np.arange(0,200),11*np.arange(0,200),'r--')
plt.xlabel('Delta time, sec')
plt.ylabel('Road Distance within the time,m')

In [286]:
# Data of Drive
plt.scatter(list_delta_sec,list_dist,color='gray',s=1)
x = np.asarray(drive_sec)
y = np.asarray(drive_dist)

# Modeling with Numpy
p, cov = np.polyfit(x,y,1,cov=True)           # parameters and covariance from of the fit
y_model = np.polyval(p, x)                    # model using the fit parameters; NOTE: parameters here are coefficients

# Statistics
n = weights.size                              # number of observations
m = p.size                                    # number of parameters
DF = n - m                                    # degrees of freedom
t = stats.t.ppf(0.95, n - m)                  # used for CI and PI bands

# Estimates of Error in Data/Model
resid = y - y_model                           
chi2 = np.sum((resid/y_model)**2)             # chi-squared; estimates error in data
chi2_red = chi2/(DF)                          # reduced chi-squared; measures goodness of fit
s_err = np.sqrt(np.sum(resid**2)/(DF))        # standard deviation of the error


# Plotting --------------------------------------------------------------------

# Data
plt.plot(x,y,'.', color='#b9cfe7', markersize=8,
         markeredgewidth=1,markeredgecolor='b',markerfacecolor='blue')

# Fit
plt.plot(x,y_model,'-', color='red', linewidth='2', alpha=0.8, label='Drive Fit')  

x2 = np.linspace(np.min(x), np.max(x), 100)
y2 = np.linspace(np.min(y_model), np.max(y_model), 100)

# Prediction Interval
PI = t*s_err*np.sqrt(1+1/n+(x2-np.mean(x))**2/np.sum((x-np.mean(x))**2))   
plt.fill_between(x2, y2+PI, y2-PI, color='None', linestyle='--')
plt.plot(x2, y2-PI, '--', color='0.5', label='95% Prediction Limits')
plt.plot(x2, y2+PI, '--', color='0.5')


# Data of Walk--------------------------------------------------------------------
x = np.asarray(walk_sec)
y = np.asarray(walk_dist)

# Modeling with Numpy
p, cov = np.polyfit(x,y,1,cov=True)           # parameters and covariance from of the fit
y_model = np.polyval(p, x)                    # model using the fit parameters; NOTE: parameters here are coefficients

# Statistics
n = weights.size                              # number of observations
m = p.size                                    # number of parameters
DF = n - m                                    # degrees of freedom
t = stats.t.ppf(0.95, n - m)                  # used for CI and PI bands

# Estimates of Error in Data/Model
resid = y - y_model                           
chi2 = np.sum((resid/y_model)**2)             # chi-squared; estimates error in data
chi2_red = chi2/(DF)                          # reduced chi-squared; measures goodness of fit
s_err = np.sqrt(np.sum(resid**2)/(DF))        # standard deviation of the error


# Plotting --------------------------------------------------------------------

# Data
plt.plot(x,y,'.', color='#b9cfe7', markersize=8,
         markeredgewidth=1,markeredgecolor='b',markerfacecolor='blue')

# Fit
plt.plot(x,y_model,'-', color='orange', linewidth='2', alpha=0.8, label='Walk Fit')  

x2 = np.linspace(np.min(x), np.max(x), 100)
y2 = np.linspace(np.min(y_model), np.max(y_model), 100)

# Prediction Interval
PI = t*s_err*np.sqrt(1+1/n+(x2-np.mean(x))**2/np.sum((x-np.mean(x))**2))   
plt.fill_between(x2, y2+PI, y2-PI, color='None', linestyle='--')
plt.plot(x2, y2-PI, '--', color='0.5', label='95% Prediction Limits')
plt.plot(x2, y2+PI, '--', color='0.5')

# Figure Modifications --------------------------------------------------------

# Labels
plt.title('Fit Plot for Distance', fontsize='14', fontweight='bold')
plt.xlabel('Travel Time (sec)')
plt.ylabel('Travel Distance (m)')
legend = plt.legend(loc=4,ncol=1)
plt.xlim(0,2500)
plt.ylim(0,3000)

# Save Figure
plt.show()

In [305]:
plt.scatter(list_delta_sec,list_dist,color='gray',s=1,label='raw data')
plt.legend(loc='lower right')
plt.xlabel('Travel Time (sec)')
plt.ylabel('Travel Distance (m)')
plt.xlim(0,2500)
plt.ylim(0,3000)

(0, 3000)

In [304]:
# Data of Drive
plt.scatter(list_delta_sec,list_dist,color='gray',s=1,label='raw data')
x = np.asarray(drive_sec)
y = np.asarray(drive_dist)

# Modeling with Numpy
p, cov = np.polyfit(x,y,1,cov=True)           # parameters and covariance from of the fit
y_model = np.polyval(p, x)                    # model using the fit parameters; NOTE: parameters here are coefficients

# Statistics
n = weights.size                              # number of observations
m = p.size                                    # number of parameters
DF = n - m                                    # degrees of freedom
t = stats.t.ppf(0.95, n - m)                  # used for CI and PI bands

# Estimates of Error in Data/Model
resid = y - y_model                           
chi2 = np.sum((resid/y_model)**2)             # chi-squared; estimates error in data
chi2_red = chi2/(DF)                          # reduced chi-squared; measures goodness of fit
s_err = np.sqrt(np.sum(resid**2)/(DF))        # standard deviation of the error


# Plotting --------------------------------------------------------------------

# Data
plt.scatter(x,y, color='gray',s=1)

# Fit
plt.plot(x,y_model,'-', color='red', linewidth='2', alpha=0.8, label='Drive Fit')  

x2 = np.linspace(np.min(x), np.max(x), 100)
y2 = np.linspace(np.min(y_model), np.max(y_model), 100)

# Prediction Interval
PI = t*s_err*np.sqrt(1+1/n+(x2-np.mean(x))**2/np.sum((x-np.mean(x))**2))   
plt.fill_between(x2, y2+PI, y2-PI, color='None', linestyle='--')
plt.plot(x2, y2-PI, '--', color='0.5', label='95% Prediction Limits')
plt.plot(x2, y2+PI, '--', color='0.5')


# Data of Walk--------------------------------------------------------------------
x = np.asarray(walk_sec)
y = np.asarray(walk_dist)

# Modeling with Numpy
p, cov = np.polyfit(x,y,1,cov=True)           # parameters and covariance from of the fit
y_model = np.polyval(p, x)                    # model using the fit parameters; NOTE: parameters here are coefficients

# Statistics
n = weights.size                              # number of observations
m = p.size                                    # number of parameters
DF = n - m                                    # degrees of freedom
t = stats.t.ppf(0.95, n - m)                  # used for CI and PI bands

# Estimates of Error in Data/Model
resid = y - y_model                           
chi2 = np.sum((resid/y_model)**2)             # chi-squared; estimates error in data
chi2_red = chi2/(DF)                          # reduced chi-squared; measures goodness of fit
s_err = np.sqrt(np.sum(resid**2)/(DF))        # standard deviation of the error


# Plotting --------------------------------------------------------------------

# Data
plt.scatter(x,y, color='gray',s=1)

# Fit
plt.plot(x,y_model,'-', color='orange', linewidth='2', alpha=0.8, label='Walk Fit')  

x2 = np.linspace(np.min(x), np.max(x), 100)
y2 = np.linspace(np.min(y_model), np.max(y_model), 100)

# Prediction Interval
PI = t*s_err*np.sqrt(1+1/n+(x2-np.mean(x))**2/np.sum((x-np.mean(x))**2))   
plt.fill_between(x2, y2+PI, y2-PI, color='None', linestyle='--')
plt.plot(x2, y2-PI, '--', color='0.5', label='95% Prediction Limits')
plt.plot(x2, y2+PI, '--', color='0.5')

# Figure Modifications --------------------------------------------------------

# Labels
plt.xlabel('Travel Time (sec)')
plt.ylabel('Travel Distance (m)')
legend = plt.legend(loc=4,ncol=1)
plt.xlim(0,2500)
plt.ylim(0,3000)

# Save Figure
plt.show()

In [303]:
# Data of Drive
plt.scatter(list_delta_sec,list_dist,color='gray',s=1,label='raw data')
x = np.asarray(drive_sec)
y = np.asarray(drive_dist)

# Modeling with Numpy
p, cov = np.polyfit(x,y,1,cov=True)           # parameters and covariance from of the fit
y_model = np.polyval(p, x)                    # model using the fit parameters; NOTE: parameters here are coefficients

# Statistics
n = weights.size                              # number of observations
m = p.size                                    # number of parameters
DF = n - m                                    # degrees of freedom
t = stats.t.ppf(0.95, n - m)                  # used for CI and PI bands

# Estimates of Error in Data/Model
resid = y - y_model                           
chi2 = np.sum((resid/y_model)**2)             # chi-squared; estimates error in data
chi2_red = chi2/(DF)                          # reduced chi-squared; measures goodness of fit
s_err = np.sqrt(np.sum(resid**2)/(DF))        # standard deviation of the error


# Plotting --------------------------------------------------------------------

# Data
plt.plot(x,y,'.', color='#b9cfe7', markersize=8,
         markeredgewidth=1,markeredgecolor='b',markerfacecolor='blue')

# Fit
plt.plot(x,y_model,'-', color='red', linewidth='2', alpha=0.8, label='Drive Fit')  

x2 = np.linspace(np.min(x), np.max(x), 100)
y2 = np.linspace(np.min(y_model), np.max(y_model), 100)

# Prediction Interval
PI = t*s_err*np.sqrt(1+1/n+(x2-np.mean(x))**2/np.sum((x-np.mean(x))**2))   
plt.fill_between(x2, y2+PI, y2-PI, color='None', linestyle='--')
plt.plot(x2, y2-PI, '--', color='0.5', label='95% Prediction Limits')
plt.plot(x2, y2+PI, '--', color='0.5')


# Data of Walk--------------------------------------------------------------------
x = np.asarray(walk_sec)
y = np.asarray(walk_dist)

# Modeling with Numpy
p, cov = np.polyfit(x,y,1,cov=True)           # parameters and covariance from of the fit
y_model = np.polyval(p, x)                    # model using the fit parameters; NOTE: parameters here are coefficients

# Statistics
n = weights.size                              # number of observations
m = p.size                                    # number of parameters
DF = n - m                                    # degrees of freedom
t = stats.t.ppf(0.95, n - m)                  # used for CI and PI bands

# Estimates of Error in Data/Model
resid = y - y_model                           
chi2 = np.sum((resid/y_model)**2)             # chi-squared; estimates error in data
chi2_red = chi2/(DF)                          # reduced chi-squared; measures goodness of fit
s_err = np.sqrt(np.sum(resid**2)/(DF))        # standard deviation of the error


# Plotting --------------------------------------------------------------------

# Data
plt.plot(x,y,'.', color='#b9cfe7', markersize=8,
         markeredgewidth=1,markeredgecolor='b',markerfacecolor='blue',label='identified trip data')

# Fit
plt.plot(x,y_model,'-', color='orange', linewidth='2', alpha=0.8, label='Walk Fit')  

x2 = np.linspace(np.min(x), np.max(x), 100)
y2 = np.linspace(np.min(y_model), np.max(y_model), 100)

# Prediction Interval
PI = t*s_err*np.sqrt(1+1/n+(x2-np.mean(x))**2/np.sum((x-np.mean(x))**2))   
plt.fill_between(x2, y2+PI, y2-PI, color='None', linestyle='--')
plt.plot(x2, y2-PI, '--', color='0.5', label='95% Prediction Limits')
plt.plot(x2, y2+PI, '--', color='0.5')

# Figure Modifications --------------------------------------------------------

# Labels
plt.xlabel('Travel Time (sec)')
plt.ylabel('Travel Distance (m)')
legend = plt.legend(loc=4,ncol=1)
plt.xlim(0,2500)
plt.ylim(0,3000)

# Save Figure
plt.show()